In [1]:
!pip install google-genai
!pip install pylatexenc

  Using cached google_genai-1.21.1-py3-none-any.whl.metadata (37 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
Using cached google_genai-1.21.1-py3-none-any.whl (206 kB)
Using cached google_auth-2.40.3-py2.py3-none-any.whl (216 kB)
Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (182 kB)
Using cached cachetools-5.5.2-py3-none-any.whl (10 kB)
Using cached pyasn1_modules-0.4.2-py3-none-any.whl (181 kB)
Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
Using cached pyasn1-0.6.1-

In [2]:
import requests

from google import genai
from google.genai import types as g_types

from helper import _finalize_prompt, _prepare_prompt_for_language, return_comparison, _ask_gemini_model, _ask_gemma_model, _ask_aristote, extract_translated_from_response

from tagging import latex_to_xml, reconstruct_from_xml

from difflib import SequenceMatcher

In [3]:

def diff_score(a: str, b: str) -> float:
    # ratio() ∈ [0..1], based on length of longest common subsequence
    return SequenceMatcher(None, a, b).ratio()

def diff_opcodes(a: str, b: str):
    # returns list of (tag, i1, i2, j1, j2)
    return SequenceMatcher(None, a, b).get_opcodes()

In [2]:
prompt = r'''
You are tasked with updating the translation of a scientific document from [SOURCE_LANGUAGE] to [TARGET_LANGUAGE] using a structured XML format.

The document consists of <TEXT> elements that contain translatable content (sentences or paragraphs), interleaved with <PH> tags that represent non-translatable content such as LaTeX commands, math expressions, or code.

### Context:
You are provided with:
1. The original source paragraph (in [SOURCE_LANGUAGE]).
2. Its correct translation (in [TARGET_LANGUAGE]).
3. A **new version of the source paragraph**, which differs only slightly (1–3 words changed).

### Your task:
- **Update the translation** to reflect the changes in the new source.
- **Reuse as much as possible** from the original translation.
- Keep the XML structure unchanged, including all <PH> tags and their attributes.

### Rules:
- Translate or modify **only the parts that changed** in the new source.
- Do **not modify, remove, or reorder** any <PH/> tags.
- Use the `original` attribute of each <PH/> tag for understanding grammar context (e.g. case, gender, plurality), but do **not translate or alter** their content.
- Your output must contain **only** the updated XML <TEXT> block — no explanations, comments, or extra markup.
- All <PH> tags must be self-closing and written in the form:
    <PH id="..." original="..."/>
- Never use closing tags like </PH> or wrap content inside <PH> tags.
- If the provided chunk doesn't contain any <PH> tags, you simply translate the text inside the <TEXT> tag and return it in the initial format

### Output Format:
<document>
<TEXT>
  ...translated text with embedded <PH id="..." original="..."/> tags...
</TEXT>
</document>

Don't cover the output in any Markdown or XML environments like (```) etc. 

### Provided Input:

#### Old Source:
[OLD_SRC]

#### Old Translation:
[OLD_TGT]

#### New Source:
[NEW_SRC]

Now provide the updated translation:
'''

In [19]:
f_old_src = r'''
Or dans notre hypothèse d'Univers de symétrie maximale, rappelons tout d'abord qu'on peut définir un temps cosmique, universel,  en utilisant l'évolution physique de l'Univers comme une horloge (densité de matière, température du CMB...). Les hypersurfaces de l'espace-temps paramétrées par ce temps universel sont alors elles-mêmes des sous-espaces de symétrie maximale. Les tenseurs $\mathcal{T}$ représentants des observables cosmologiques de tels sous-espaces de symétrie maximale doivent alors être de _forme invariante_ c'est-à-dire qu'ils restent les mêmes fonctions des coordonnées spatiales à une date $t$ quelque soit le choix du système de coordonnées choisi : si on passe d'un système $x^\rho$ à $x'^\rho$, on doit avoir $\mathcal{T}'_{\mu\nu\ldots}(x'^\rho) = \mathcal{T}_{\mu\nu\ldots}(x'^\rho)$. Intuitivement, si $\mathcal{T}$ est le tenseur énergie-impulsion cela revient entre autre à demander que la densité d'énergie soit identique en tout point pour tout choix de système de coordonnées {cite:p}`Weinberg1972`[p. 409]. On peut démontrer alors une propriété importante concernant la forme que doivent prendre les tenseurs de ces sous-espaces {cite:p}`Weinberg1972`[p. 392].
'''


In [8]:
huy = r'''Enfin, l’algèbre linéaire numérique repose largement sur la décomposition LU et la factorisation QR'''

In [4]:
old_tgt = r'''
Now, in our hypothesis of a Universe of maximum symmetry, let's first recall that we can define a cosmic, universal time, using the physical evolution of the Universe as a clock (matter density, CMB temperature...). The hypersurfaces of space-time parametrized by this universal time are then themselves subspaces of maximum symmetry. The $\mathcal{T}$ tensors representing the cosmological observables of such maximally symmetric subspaces must then be of _form invariant_, i.e. they remain the same functions of the spatial coordinates at a date $t$ whatever the chosen coordinate system: if we go from a $x^\rho$ system to $x'^\rho$, we must have $\mathcal{T}'_{\mu\nu\ldots}(x'^\rho) = \mathcal{T}_{\mu\nu\ldots}(x'^\rho)$. Intuitively, if $\mathcal{T}$ is the energy-momentum tensor, this means, among other things, that the energy density must be identical at all points for any choice of coordinate system {cite:p}`Weinberg1972`[p. 409]. We can then demonstrate an important property concerning the form that the tensors of these subspaces {cite:p}`Weinberg1972`[p. 392] must take.
'''

In [5]:
new_src = r'''
Or dans notre hypothèse d’Univers de symétrie maximale, rappelons tout d’abord qu’on peut définir un temps cosmique, universel, en utilisant l’évolution physique de l’Univers comme une horloge (densité de matière, température du CMB…). Les hypersurfaces de l’espace-temps paramétrées par ce temps universel sont alors elles-mêmes des sous-espaces de symétrie maximale. Les tenseurs $\mathcal{T}$ représentants des observables cosmologiques de tels sous-espaces de symétrie maximale doivent alors être de forme invariante c’est-à-dire qu’ils restent les mêmes fonctions des coordonnées spatiales à une date $t$ quelque soit le choix du système de coordonnées choisi : si on passe d’un système $x^\rho$ à $x’^\rho$, on doit avoir $\mathcal{T}’{\mu\nu\ldots}(x’^\rho) = \mathcal{T}{\mu\nu\ldots}(x’^\rho)$. Intuitivement, si $\mathcal{T}$ est le tenseur énergie-impulsion, cela revient entre autre à demander que la densité d’énergie soit identique en tout point pour tout choix de système de coordonnées {cite:p}Weinberg1972[p. 409]. Cette propriété implique également que les lois de conservation physiques associées à ces tenseurs, comme celles de l’énergie ou de la quantité de mouvement, s’expriment de manière identique quelle que soit la position ou l’orientation de l’observateur. On peut démontrer alors une propriété importante concernant la forme que doivent prendre les tenseurs de ces sous-espaces {cite:p}Weinberg1972[p. 392].
'''


In [6]:
tgt_lang = "English"
src_lang = "French"

In [14]:
(old_src, old_tgt) = (r'''Enfin, l’algèbre linéaire numérique repose largement sur la
décomposition LU et la factorisation QR, qui permettent de résoudre
efficacement des systèmes linéaires et de calculer des valeurs
propres.
\Sage intègre des interfaces vers les bibliothèques LAPACK et BLAS,
assurant des performances comparables aux programmes spécialisés.''',
 r'''Numerical linear algebra relies heavily on LU decomposition and QR
factorisation, which efficiently solve linear systems and compute
eigenvalues.  \Sage interfaces to the LAPACK and BLAS libraries,
providing performance on par with dedicated programs.''')

new_src = r'''Enfin, l’algèbre linéaire numérique repose largement sur la
décomposition LU et la factorisation QR (deux méthodes fondamentales dans l'algèbre linéaire), qui permettent de résoudre
efficacement des systèmes linéaires et de calculer des valeurs et vecteurs
propres.
\Sage intègre des interfaces vers les bibliothèques LAPACK et BLAS,
assurant des performances comparables aux programmes spécialisés.'''

In [22]:
diff_score("Idi nahuy, l'algèbre linéaire", new_src)
# diff_opcodes(old_src, new_src)

0.014018691588785047

In [17]:
old_src_xml = latex_to_xml(old_src)[0]
old_tgt_xml = latex_to_xml(old_tgt)[0]

In [18]:
old_src_xml

'<document><TEXT>Enfin, l’algèbre linéaire numérique repose largement sur la\ndécomposition LU et la factorisation QR, qui permettent de résoudre\nefficacement des systèmes linéaires et de calculer des valeurs\npropres.\n<PH id="1" original="\\Sage " />intègre des interfaces vers les bibliothèques LAPACK et BLAS,\nassurant des performances comparables aux programmes spécialisés.</TEXT></document>'

### gemini translation

In [19]:
new_src_xml, new_src_ph = latex_to_xml(new_src)

In [20]:
new_src_ph

{'1': '\\Sage '}

In [21]:
final_prompt = prompt.replace("[OLD_SRC]", old_src_xml).replace("[OLD_TGT]", old_tgt_xml).replace("[NEW_SRC]", new_src).replace("[TARGET_LANGUAGE]", tgt_lang).replace("[SOURCE_LANGUAGE]", src_lang)

In [22]:
res = await _ask_gemini_model(final_prompt, "gemini-2.0-flash")

In [23]:
res

'<document><TEXT>Numerical linear algebra relies heavily on LU decomposition and QR\nfactorisation, which efficiently solve linear systems and compute\neigenvalues and eigenvectors.  <PH id="1" original="\\Sage " />interfaces to the LAPACK and BLAS libraries,\nproviding performance on par with dedicated programs.</TEXT></document>\n'

In [24]:
reconstruct_from_xml(res, new_src_ph)

'Numerical linear algebra relies heavily on LU decomposition and QR\nfactorisation, which efficiently solve linear systems and compute\neigenvalues and eigenvectors.  \\Sage interfaces to the LAPACK and BLAS libraries,\nproviding performance on par with dedicated programs.'

### aristote translation

In [38]:
res = await _ask_aristote(final_prompt)

In [39]:
reconstruct_from_xml(res, new_src_ph)

"\nNow, in our hypothesis of a Universe of maximum symmetry, let's first recall that we can define a cosmic, universal time, using the physical evolution of the Universe as a clock (matter density, CMB temperature...). The hypersurfaces of space-time parametrized by this universal time are then themselves subspaces of maximum symmetry. The $\\mathcal{T}$ tensors representing the cosmological observables of such maximally symmetric subspaces must then be of form invariant, i.e. they remain the same functions of the spatial coordinates at a date $t$ whatever the chosen coordinate system: if we go from a $x^\\rho$ system to $x’^\\rho$, we must have $\\mathcal{T}’{\\mu\\nu\\ldots}(x’^\\rho) = \\mathcal{T}{\\mu\\nu\\ldots}(x’^\\rho)$. Intuitively, if $\\mathcal{T}$ is the energy-momentum tensor, this means, among other things, that the energy density must be identical at all points for any choice of coordinate system {cite:p}`Weinberg1972`[p. 409]. This property also implies that the physic

In [27]:
async def usual_translation(chunk: str, src_lang: str, tgt_lang: str) -> str:
    local_prompt = r'''
    You are tasked with translating scientific text from [SOURCE_LANGUAGE] to [TARGET_LANGUAGE] using a structured XML format.
    
    The document is composed of <TEXT> elements that contain the full translatable content (sentences or paragraphs), interleaved with <PH> tags for non-translatable content such as LaTeX commands, math expressions, or code.
    Instructions:
        - Translate only the content inside <TEXT> tags, excluding anything inside <PH> tags.
        - Do not remove, modify any <PH/> tags or their attributes.
        - Use the original attribute of each <PH/> tag to understand the context and grammar. This will help you make correct translation decisions (e.g., for plurality, case, or syntax), but you must not change or translate the contents of the <PH> tags themselves.
        - Treat each <TEXT> block as a complete sentence or paragraph. You may reorder words, adjust structure, and apply natural grammar in the target language — as long as all <PH> tags remain in place and unchanged.
        - Your response must contain only the translated XML — return the modified <TEXT> block with embedded <PH> tags and nothing else (no explanations, no markdown, no prefix/suffix text).
        - All <PH> tags must be self-closing and written in the form: 
            <PH id="..." original="..."/>
        - Do not produce </PH> closing tags, and do not place content inside <PH> elements. Any other structure is invalid and will break XML parsing.
        - If the provided chunk doesn't contain any <PH> tags, you simply translate the text inside the <TEXT> tag and return it in the initial format.
            Example (Spanish to Ukrainian):
                Input:
                ```
                <document><TEXT>El gato duerme en la silla.</TEXT><document>
                ```
                Output:
                ```
                <document><TEXT>Кіт спить на стільці.</TEXT><document>
                ```
    Output Format:
    <document>
    <TEXT>
      ...translated text and inline <PH id="..." original="..."/> tags (if such presented in the input)...
    </TEXT>
    </document>
    
    Don't cover the output in any Markdown or XML environments like (```) etc. 
    
    The document is provided below:
    '''
    local_prompt = local_prompt.replace("[TARGET_LANGUAGE]", tgt_lang).replace("[SOURCE_LANGUAGE]", src_lang)
    local_prompt += chunk
    return await _ask_gemini_model(local_prompt, "gemini-2.0-flash")

In [40]:
old_src

"\nOr dans notre hypothèse d'Univers de symétrie maximale, rappelons tout d'abord qu'on peut définir un temps cosmique, universel,  en utilisant l'évolution physique de l'Univers comme une horloge (densité de matière, température du CMB...). Les hypersurfaces de l'espace-temps paramétrées par ce temps universel sont alors elles-mêmes des sous-espaces de symétrie maximale. Les tenseurs $\\mathcal{T}$ représentants des observables cosmologiques de tels sous-espaces de symétrie maximale doivent alors être de _forme invariante_ c'est-à-dire qu'ils restent les mêmes fonctions des coordonnées spatiales à une date $t$ quelque soit le choix du système de coordonnées choisi : si on passe d'un système $x^\\rho$ à $x'^\\rho$, on doit avoir $\\mathcal{T}'_{\\mu\\nu\\ldots}(x'^\\rho) = \\mathcal{T}_{\\mu\\nu\\ldots}(x'^\\rho)$. Intuitivement, si $\\mathcal{T}$ est le tenseur énergie-impulsion cela revient entre autre à demander que la densité d'énergie soit identique en tout point pour tout choix d

In [41]:
old_src_xml, old_src_ph = latex_to_xml(old_src)

In [30]:
await usual_translation(old_src_xml, "French", "English")

'<document><TEXT>In our hypothesis of a Universe with maximal symmetry, let us first recall that we can define a cosmic, universal time, by using the physical evolution of the Universe as a clock (matter density, CMB temperature, etc.). The space-time hypersurfaces parameterized by this universal time are then themselves subspaces of maximal symmetry. The tensors <PH id="1" original="$\\mathcal{T}$"/> representing cosmological observables of such subspaces of maximal symmetry must then be of _invariant form_, that is to say that they remain the same functions of the spatial coordinates at a date <PH id="2" original="$t$"/> regardless of the choice of the coordinate system chosen: if we move from a system <PH id="3" original="$x^\\rho$"/> to <PH id="4" original="$x\'^\\rho$"/>, we must have <PH id="5" original="$\\mathcal{T}\'_{\\mu\\nu\\ldots}(x\'^\\rho) = \\mathcal{T}_{\\mu\\nu\\ldots}(x\'^\\rho)$"/>. Intuitively, if <PH id="6" original="$\\mathcal{T}$"/> is the energy-momentum tensor